<a href="https://colab.research.google.com/github/marreapato/Forecasting_Paper_XAI/blob/main/Times_cv/Tcrossvalidation_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
import math
import yfinance as yf
# Assuming you have loaded your time series data from Yahoo Finance
# For demonstration purposes, let's create a dummy time series
# Replace this with loading your actual data

bvsp_data = yf.download('^BVSP', '2007-01-01', '2023-10-17')

# Example with dummy data
np.random.seed(42)
#dates = pd.date_range(start="2007-01-01", end="2023-10-17", freq="D")
#bvsp_data = pd.DataFrame({"Close": np.random.rand(len(dates))}, index=dates)

bvsp_datancls=bvsp_data#.drop(['Close'],axis = 1)
bvsp_data = bvsp_data[['Close']]
#bvsp_data.reset_index(inplace=True)

# Feature scaling
scaler = MinMaxScaler()
bvsp_data_scaled = scaler.fit_transform(bvsp_data)
bvsp_data_scaledncls = scaler.fit_transform(bvsp_datancls)

# Assuming you have a time series array and corresponding labels
X = bvsp_data_scaledncls
y = bvsp_data_scaled

# Function to create LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=128, activation='relu', input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Define the LSTM model
def biLSTM_Model(input_shape):
    # n_steps = x_train.shape[-2]
    # n_features = x_train.shape[-1]
    model = Sequential([
        tf.keras.layers.Reshape((1, -1)),
        tf.keras.layers.Bidirectional(
          tf.keras.layers.LSTM(128, activation='relu', input_shape=input_shape, return_sequences=True)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)]
                       )
    # model.add(tf.keras.layers.Embedding(1, 6, mask_zero=True))


    ##model.add(Bidirectional(LSTM(128, activation='relu', input_shape=(1, look_back), return_sequences=False)))
    #model.add(Bidirectional(LSTM(32, activation='relu', return_sequences=False)))
    #model.add(Bidirectional(LSTM(16, activation='relu')))
    # model.add(LSTM(128, return_sequences=False, input_shape=(1, look_back)))
    #model.add(Dense(output_window))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


# Define the GRU model
def GRU_Model(input_shape):
    model = Sequential([
            tf.keras.layers.Reshape((1, -1)),
            tf.keras.layers.GRU(128, return_sequences=False, input_shape=input_shape),
            tf.keras.layers.Dense(1)
        ])
#    model.add(GRU(128, return_sequences=False, input_shape=(1, look_back)))
 #   model.add(Dense(output_window))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model



# Define the baseline model
def baseline_model(output_window):
    last_known_value = trainY[-1]
    return np.full((len(testY), output_window), last_known_value)


[*********************100%%**********************]  1 of 1 completed


In [14]:
y

array([[0.15736967],
       [0.14811379],
       [0.14392003],
       ...,
       [0.86456617],
       [0.85176779],
       [0.85946458]])

In [15]:
X

array([[0.14839448, 0.13948451, 0.15027325, 0.15736967, 0.15736967,
        0.3253185 ],
       [0.15730526, 0.13939424, 0.148355  , 0.14811379, 0.14811379,
        0.52686648],
       [0.14805897, 0.12999699, 0.14175101, 0.14392003, 0.14392003,
        0.37275047],
       ...,
       [0.86146362, 0.85868017, 0.86717087, 0.86456617, 0.86456617,
        0.04283808],
       [0.86455229, 0.85838933, 0.86144608, 0.85176779, 0.85176779,
        0.04567751],
       [0.85182261, 0.85673453, 0.86246516, 0.85946458, 0.85946458,
        0.03870531]])

In [16]:
len(X)

4154

In [17]:
# Time series cross-validation using TimeSeriesSplit
n_splits = 30  # You can adjust the number of splits
tscv = TimeSeriesSplit(n_splits=n_splits)
tscv


TimeSeriesSplit(gap=0, max_train_size=None, n_splits=30, test_size=None)

In [18]:
for train_index, test_index in tscv.split(X):
      print("TRAIN:", train_index, "TEST:", test_index)


TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133] TEST: [134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169
 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187
 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205
 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223
 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241
 242

In [19]:
resultslstm = []
resultsbilstm = []
resultsgru = []

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Reshape the data for LSTM input
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    # Create and train the LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    bilstm_model = biLSTM_Model(input_shape)
    model = create_lstm_model(input_shape)
    gru_model = GRU_Model(input_shape)
    model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=0)

    bilstm_model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=0)
    gru_model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=0)

    # Evaluate the model on the test set
    mse = model.evaluate(X_test, y_test, verbose=0)
    bmse = bilstm_model.evaluate(X_test, y_test, verbose=0)
    gmse = gru_model.evaluate(X_test, y_test, verbose=0)
    resultslstm.append(math.sqrt(mse))
    resultsbilstm.append(math.sqrt(bmse))
    resultsgru.append(math.sqrt(gmse))
    print(f'Mean Squared Error for this fold: {math.sqrt(mse)}')

Mean Squared Error for this fold: 0.268956372955431
Mean Squared Error for this fold: 0.28158768938117223
Mean Squared Error for this fold: 0.09101339550316487
Mean Squared Error for this fold: 0.12980516163282793
Mean Squared Error for this fold: 0.23871585422022415
Mean Squared Error for this fold: 0.21460808052641886
Mean Squared Error for this fold: 0.18734836407245745
Mean Squared Error for this fold: 0.0951185717863833
Mean Squared Error for this fold: 0.08889733840547234
Mean Squared Error for this fold: 0.04802228678374525
Mean Squared Error for this fold: 0.018943896468078734
Mean Squared Error for this fold: 0.040676105316455234
Mean Squared Error for this fold: 0.028777232719026994
Mean Squared Error for this fold: 0.03365987532362193
Mean Squared Error for this fold: 0.036420247537136095
Mean Squared Error for this fold: 0.060048179964174025
Mean Squared Error for this fold: 0.02540562832907238
Mean Squared Error for this fold: 0.028213766085429786
Mean Squared Error for th

In [20]:
np.mean(resultslstm)

0.08244510420378304

In [21]:
np.mean(resultsbilstm)

0.0589497633327114

In [22]:
np.mean(resultsgru)

0.037019491079640233

# Avaliação estatística

In [27]:
from scipy.stats import shapiro

#make this example reproducible
np.random.seed(1)

#perform Shapiro-Wilk test for normality
print(shapiro(resultsbilstm),"\n",
shapiro(resultslstm),"\n",
shapiro(resultsgru))#no evidence to say that data follows a normal distribution, hence we reject the null hypothesis going for non parametrical test

ShapiroResult(statistic=0.7046726942062378, pvalue=1.825925096454739e-06) 
 ShapiroResult(statistic=0.7713913917541504, pvalue=2.0587398466886953e-05) 
 ShapiroResult(statistic=0.7211340665817261, pvalue=3.215777724108193e-06)
